***
# 1.) Evaluate on HateCheck
## Load HateCheck

In [7]:
#PR: DG, please adapt this function to load HateCheck and return it in the correct format for further evaluation.
#PR: There is a lot of unneccessary code in here. We are loading HateCheck as a whole and it would make things more clear if the var names reflected that

def LoadHateCheck():

    """
    Function to load HateCheck data
    """
    
    df_train, df_valtest, df_val, df_test = {}, {}, {}, {}
    test_texts, test_labels={},{}
    test_encodings = {}
    test_dataset = {}
    
    if len(dataset) == 1:


        #df_test[dataset]=df_raw


        #print(df_raw[dataset])
        for dset in dataset:
            test_labels[dset] = df_raw.label.tolist()
            test_texts[dset] = df_raw.text.astype("string").tolist()
            #print(test_labels[dset])
            #print(test_texts[dset])
            test_encodings[dset] = tokenizer(test_texts[dset], truncation=True, padding=True)
            #print(test_encodings[dset])
            test_dataset[dset] = HateDataset(test_encodings[dset], test_labels[dset])

        return test_dataset
    else : 
        #print(df_raw[dataset])
        for dset in dataset:
            test_labels[dset] = df_raw[dset].label.tolist()
            test_texts[dset] = df_raw[dset].text.astype("string").tolist()
            #print(test_labels[dset])
            #print(test_texts[dset])
            test_encodings[dset] = tokenizer(test_texts[dset], truncation=True, padding=True)
            #print(test_encodings[dset])
            test_dataset[dset] = HateDataset(test_encodings[dset], test_labels[dset])

        return test_dataset
        



In [13]:
#PR: Hatecheck should be loaded in the previous function, not this one.
def evaluate_on_hatecheck(hatecheck_path ="./Data/Test Suite/hatecheck_final_ACL.csv"):
    hatecheck_df = pd.read_csv(hatecheck_path, index_col=0)
    dataframe = hatecheck_df.copy()
    dataframe.rename({"test_case":"text","label_gold":"label"},axis = 1,inplace = True)
    dataframe.label.replace({'hateful': 1, 'non-hateful': 0}, inplace = True)
    case_id = dataframe.case_id.tolist()
    trainer = getModels(["combined"])
     
    test_dataset = setupHatecheckTestDataset(df_raw = dataframe)
    
    
    #print(test_dataset["combined"])
    
    #print(trainer["combined"].predict(test_dataset["combined"]))
    
    result_df = getModelPrediction(trainer, test_dataset,index=case_id)
    
    result_df["combined"] = result_df["combined"].reset_index().rename({"index":"case_id"},axis = 1)
    result_df["combined"].pred_BERT_combined_weighted.replace({1:"hateful",0:"non-hateful"},inplace = True)
    hatecheck_df = hatecheck_df.merge(result_df["combined"], how = 'right', on = 'case_id')
    return hatecheck_df
    #print()
hatecheck_df = evaluate_on_hatecheck()

combined
Evaluating weighted combined BERT model on test data


## Compute Accuracy by Functionality across Models

In [14]:
# set of models to evaluate
models_eval = ["BERT_combined_weighted"]

In [16]:
# PR: except for the pred_ column, this dataframe should be the result of "loadHateCheck"
# PR: could we follow the same format as for the test sets? i.e. load HateCheck in one function, then get and concatenate model predictions on HateCheck in the next?
hatecheck_df

,functionality,case_id,test_case,label_gold,target_ident,direction,focus_words,focus_lemma,ref_case_id,ref_templ_id,templ_id,case_templ,pred_BERT_combined_weighted
0,derog_neg_emote_h,1,I hate women.,hateful,women,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,non-hateful
1,derog_neg_emote_h,2,I hate trans people.,hateful,trans people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,non-hateful
2,derog_neg_emote_h,3,I hate gay people.,hateful,gay people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,non-hateful
3,derog_neg_emote_h,4,I hate black people.,hateful,black people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,non-hateful
4,derog_neg_emote_h,5,I hate disabled people.,hateful,disabled people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,non-hateful
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3723,spell_leet_h,3897,Gay people are all complete 4ssholes.,hateful,gay people,general,4ssholes,asshole,1232.0,256.0,866,[IDENTITY_P] are all complete 4ssholes.,non-hateful
3724,spell_leet_h,3898,Black people are all complete 4ssholes.,hateful,black people,general,4ssholes,asshole,1233.0,256.0,866,[IDENTITY_P] are all complete 4ssholes.,non-hateful
3725,spell_leet_h,3899,Disabled people are all complete 4ssholes.,hateful,disabled people,general,4ssholes,asshole,1234.0,256.0,866,[IDENTITY_P] are all complete 4ssholes.,non-hateful
3726,spell_leet_h,3900,Muslims are all complete 4ssholes.,hateful,Muslims,general,4ssholes,asshole,1235.0,256.0,866,[IDENTITY_P] are all complete 4ssholes.,non-hateful


In [17]:
# write data to dict
func_accuracy_dict = {}

for m in models_eval:
    #print("m")
    func_accuracy_dict[m] = []
    for func in pd.unique(hatecheck_df.functionality):
        #print("func")
        n_cases = hatecheck_df[hatecheck_df.functionality==func].shape[0]
        #print(n_cases)
        #print(hatecheck_df[(hatecheck_df.functionality==func)&(hatecheck_df['label_gold']==hatecheck_df['pred_{}'.format(m)])])
        n_correct = hatecheck_df[(hatecheck_df.functionality==func)&(hatecheck_df['label_gold']==hatecheck_df['pred_{}'.format(m)])].shape[0]
        func_accuracy_dict[m].append('{:.1%}'.format(n_correct/n_cases))
    
    # convert list to series
    func_accuracy_dict[m] = pd.Series(func_accuracy_dict[m])
    func_accuracy_dict[m].name = m

In [18]:
# create df from dict
func_accuracy_df = pd.Series(pd.unique(hatecheck_df.functionality))
func_accuracy_df.name = 'functionality'

for arc_data in func_accuracy_dict:
    func_accuracy_df = pd.concat([func_accuracy_df, pd.Series(func_accuracy_dict[arc_data])], axis =1)

In [19]:
func_accuracy_df

,functionality,BERT_combined_weighted
0,derog_neg_emote_h,8.6%
1,derog_neg_attrib_h,27.9%
2,derog_dehum_h,25.7%
3,derog_impl_h,47.9%
4,threat_dir_h,20.3%
5,threat_norm_h,17.1%
6,slur_h,11.1%
7,slur_homonym_nh,96.7%
8,slur_reclaimed_nh,88.9%
9,profanity_h,12.1%


## Compute Classification Reports for Each Model 

In [20]:
# print classification reports for each classifier

for m in models_eval:
        print(m.upper())
        print(classification_report(hatecheck_df.label_gold.replace({'hateful': 1, 'non-hateful':0}),
                                    hatecheck_df['pred_{}'.format(m)].replace({'hateful': 1, 'non-hateful':0}),
                                    target_names = ['non-hateful','hateful']))
        print()

BERT_COMBINED_WEIGHTED
              precision    recall  f1-score   support

 non-hateful       0.29      0.74      0.42      1165
     hateful       0.62      0.19      0.29      2563

    accuracy                           0.36      3728
   macro avg       0.46      0.47      0.36      3728
weighted avg       0.52      0.36      0.33      3728




In [21]:
print('ACCURACY \n')
for m in models_eval:
    print(m)
    for lab in ['hateful', 'non-hateful']:
        n_cases = hatecheck_df[hatecheck_df.label_gold==lab].shape[0]
        n_correct = hatecheck_df[(hatecheck_df.label_gold==lab) & (hatecheck_df['label_gold']==hatecheck_df['pred_{}'.format(m)])].shape[0]
        print('{}: {:.1%}:'.format(lab, n_correct/n_cases))
    print('TOTAL: {:.1%}'.format(hatecheck_df[(hatecheck_df['label_gold']==hatecheck_df['pred_{}'.format(m)])].shape[0]/hatecheck_df.shape[0]))
    print()

ACCURACY 

BERT_combined_weighted
hateful: 18.9%:
non-hateful: 74.2%:
TOTAL: 36.2%



## Performance for Reclaimed Slurs

In [22]:
hatecheck_df[(hatecheck_df.functionality == 'slur_reclaimed_nh')].groupby('focus_lemma').case_id.count().keys()

print('ACCURACY')
for m in models_eval:
    print(m.upper())
    for slur in ['nigga', 'fag', 'faggot', 'queer', 'bitch']:
        n_total = hatecheck_df[(hatecheck_df.functionality == 'slur_reclaimed_nh')&
                                (hatecheck_df.focus_lemma==slur)].shape[0]
        n_correct = hatecheck_df[(hatecheck_df.functionality == 'slur_reclaimed_nh')&
                                  (hatecheck_df['pred_{}'.format(m)]==hatecheck_df.label_gold)&
                                  (hatecheck_df.focus_lemma==slur)].shape[0]
        print(n_total)
        print(slur, ': {:.1%}'.format(n_correct/n_total))
    print()
    

ACCURACY
BERT_COMBINED_WEIGHTED
19
nigga : 100.0%
16
fag : 87.5%
16
faggot : 100.0%
15
queer : 53.3%
15
bitch : 100.0%



## Performance Across Cases by Target Identity
Only uses cases generated from templates

In [23]:
# create df with only template cases --> number of cases for each identity should be balanced
templ_cases_df = hatecheck_df[hatecheck_df.case_templ.str.contains('IDENTITY')].copy()

templ_cases_df.groupby(templ_cases_df.target_ident).case_id.count()

target_ident
Muslims            421
black people       421
disabled people    421
gay people         421
immigrants         421
trans people       421
women              421
Name: case_id, dtype: int64

In [24]:
# write data to dict
ident_accuracy_dict = {}

for m in models_eval:
    ident_accuracy_dict[m] = []
    for ident in pd.unique(templ_cases_df.target_ident):
        n_cases = templ_cases_df[templ_cases_df.target_ident==ident].shape[0]
        n_correct = templ_cases_df[(templ_cases_df.target_ident==ident)&(templ_cases_df['label_gold']==templ_cases_df['pred_{}'.format(m)])].shape[0]
        ident_accuracy_dict[m].append('{:.1%}'.format(n_correct/n_cases))
    ident_accuracy_dict[m] = pd.Series(ident_accuracy_dict[m])
    ident_accuracy_dict[m].name = m

In [25]:
# create df from dict
ident_accuracy_df = pd.Series(pd.unique(templ_cases_df.target_ident))
ident_accuracy_df.name = 'target_ident'

for arc_data in ident_accuracy_dict:
    ident_accuracy_df = pd.concat([ident_accuracy_df, pd.Series(ident_accuracy_dict[arc_data])], axis =1)

In [26]:
ident_accuracy_df

,target_ident,BERT_combined_weighted
0,women,27.8%
1,trans people,26.8%
2,gay people,25.2%
3,black people,26.4%
4,disabled people,39.0%
5,Muslims,36.1%
6,immigrants,26.8%
